In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
// Class switching example 1
// A basic M/M/1 with explicit definition of a ClassSwitch node
// Recommended ClassSwitch declaration style
val model = Network("mm1cs")

In [ ]:
// Block 1: nodes
val node = arrayOfNulls<Node>(4)
node[0] = Source(model, "Source 1")
node[1] = Queue(model, "Queue 1", SchedStrategy.FCFS)
node[2] = Sink(model, "Sink 1")
node[3] = ClassSwitch(model, "ClassSwitch 1")

// Block 2: classes
val jobclass = arrayOfNulls<OpenClass>(2)
jobclass[0] = OpenClass(model, "Class1", 0)
jobclass[1] = OpenClass(model, "Class2", 0)

In [ ]:
// Service configurations
node[0]!!.setArrival(jobclass[0]!!, Exp.fitMean(10.0))  // (Source 1,Class1)
node[0]!!.setArrival(jobclass[1]!!, Exp.fitMean(2.0))   // (Source 1,Class2)
node[1]!!.setService(jobclass[0]!!, Exp.fitMean(1.0))   // (Queue 1,Class1)
node[1]!!.setService(jobclass[1]!!, Exp.fitMean(1.0))   // (Queue 1,Class2)

In [ ]:
// Block 3: topology
// The class switching matrix can now be declared after the classes, so the
// ClassSwitch node can be declared outside Block 1.
val csmatrix = (node[3] as ClassSwitch).initClassSwitchMatrix()  // element (i,j) = probability that class i switches to j

// Get class indices for the matrix
val class1Idx = jobclass[0]!!.index
val class2Idx = jobclass[1]!!.index

// Set class switching probabilities using matrix indexing
csmatrix.set(class1Idx, class1Idx, 0.3)
csmatrix.set(class1Idx, class2Idx, 0.7)
csmatrix.set(class2Idx, class1Idx, 1.0)
(node[3] as ClassSwitch).setClassSwitchingMatrix(csmatrix)

val P = model.initRoutingMatrix()  // initialize routing matrix
P.set(jobclass[0], jobclass[0], node[0], node[3], 1.0)  // (Source 1,Class1) -> (ClassSwitch 1,Class1)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)  // (Queue 1,Class1) -> (Sink 1,Class1)
P.set(jobclass[0], jobclass[0], node[3], node[1], 1.0)  // (ClassSwitch 1,Class1) -> (Queue 1,Class1)
P.set(jobclass[1], jobclass[1], node[0], node[3], 1.0)  // (Source 1,Class2) -> (ClassSwitch 1,Class2)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)  // (Queue 1,Class2) -> (Sink 1,Class2)
P.set(jobclass[1], jobclass[1], node[3], node[1], 1.0)  // (ClassSwitch 1,Class2) -> (Queue 1,Class2)
model.link(P)

In [ ]:
// Solve
val solver = MVA(model)
val avgTable = solver.avgChainTable
avgTable.print()